# **AutoKeras Regression**

In [ ]:
pip install autokeras

In [ ]:
from numpy import asarray
from pandas import read_csv
from sklearn.model_selection import train_test_split
from autokeras import StructuredDataRegressor

## Load Dataset

In [ ]:
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/auto-insurance.csv'
dataframe = read_csv(url, header=None)
print(dataframe.shape)

(63, 2)


In [ ]:
dataframe.head()

,0,1
0,108,392.5
1,19,46.2
2,13,15.7
3,124,422.2
4,40,119.4


## Train/Test Split

In [ ]:
# split into input and output elements
data = dataframe.values
data = data.astype('float32')
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)

(63, 1) (63,)


In [ ]:
# separate into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(42, 1) (21, 1) (42,) (21,)


## Define Search

In [ ]:
# define the search
search = StructuredDataRegressor(max_trials=15, loss='mean_absolute_error')
# perform the search
search.fit(x=X_train, y=y_train, verbose=0)

INFO:tensorflow:Reloading Oracle from existing project ./structured_data_regressor/oracle.json
INFO:tensorflow:Reloading Tuner from ./structured_data_regressor/tuner0.json
INFO:tensorflow:Oracle triggered exit
INFO:tensorflow:Assets written to: ./structured_data_regressor/best_model/assets


## Evaluate the Model

In [ ]:
# evaluate the model
mae, _ = search.evaluate(X_test, y_test, verbose=0)
print('MAE: %.3f' % mae)

MAE: 25.441


## Prediction

In [ ]:
# use the model to make a prediction
X_new = asarray([[108]]).astype('float32')
yhat = search.predict(X_new)
print('Predicted: %.3f' % yhat[0])

Predicted: 394.949


## Model Summary

In [ ]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
multi_category_encoding (Mul (None, 1)                 0         
_________________________________________________________________
dense (Dense)                (None, 32)                64        
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
regression_head_1 (Dense)    (None, 1)                 33        
Total params: 97
Trainable params: 97
Non-trainable params: 0
_________________________________________________________________


## Save the Model

In [ ]:
# save the best performing model to file
model.save('model_insurance.h5')